In [ ]:
# for adding the videos to DB
# don't use at the same time with the server running
# https://stackoverflow.com/questions/59119396/how-to-use-django-3-0-orm-in-a-jupyter-notebook-without-triggering-the-async-con
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

#from backend.ml_models import DatabasePreferenceLearner
import numpy as np

%load_ext autoreload
%autoreload 2

from backend.ml_model.client_server.django_ml_featureless import DatabasePreferenceLearnerFeatureless
from backend.ml_model.helpers import convert_to_tf

from backend.rating_fields import VIDEO_FIELDS
from backend.models import VideoRating, UserPreferences

import tensorflow as tf

from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
[[getattr(x, f) for f in VIDEO_FIELDS] for x in VideoRating.objects.filter(video__video_id="gxAaO2rsdIs")]

In [ ]:
l = DatabasePreferenceLearnerFeatureless()
l.fit()

In [ ]:
mb = l.aggregator.sample_minibatch(1000, 1000, 1000)

In [ ]:
l.all_ratings.aggregate_index

In [ ]:
locals().update(mb)

In [ ]:
self = l.aggregator

In [ ]:
v_int_id = self.all_ratings.objects_reverse['gxAaO2rsdIs']

In [ ]:
v_int_id

In [ ]:
C = 1
mu = 1e-2
lambda_ = 1e-2

In [ ]:
theta_s_withcoeff[objects_all == 44]

In [ ]:
idx_all[num_ratings_all > 0]

In [ ]:
result = {}

# internal indices for experts and objects (ratings)
idx_v1 = tf.stack((experts_rating, objects_rating_v1), axis=1)
idx_v2 = tf.stack((experts_rating, objects_rating_v2), axis=1)

# 2D array (comparison_id, feature) -> float
theta_eqv = self.all_ratings.model(idx_v1)
theta_eqw = self.all_ratings.model(idx_v2)

# FIT LOSS SUM
theta_vw = theta_eqv - theta_eqw
# print(theta_vw.shape, cmp.shape)
theta_vw_y = tf.math.multiply(theta_vw, cmp)
sp = tf.math.softplus(theta_vw_y)
sp_weighted = tf.math.multiply(sp, weights)

result['loss_fit'] = tf.reduce_sum(sp_weighted)

# LOSS MODEL TO COMMON
# common expert
experts_common = self.all_ratings.aggregate_index * tf.ones(shape=experts_all.shape, dtype=tf.int64)

# indices for experts for regularization
idx_all = tf.stack((experts_all, objects_all), axis=1)
idx_common = tf.stack((experts_common, objects_all), axis=1)

# 2D array (regul_id, feature) -> float
theta_eqv_common = self.all_ratings.model(idx_all)
s_qv = self.all_ratings.model(idx_common)

# print("IDX", idx_common.shape, s_qv.shape)

# coefficient, shape: regul_id
num_float = tf.cast(num_ratings_all, tf.keras.backend.floatx())
coef_yev = tf.divide(num_float, C + num_float)
# print(theta_eqv_common.shape, s_qv.shape)
theta_s = tf.abs(theta_eqv_common - s_qv)
coef_yev_repeated = tf.repeat(tf.expand_dims(coef_yev, axis=1), theta_s.shape[1], axis=1)
# print("THETAS", theta_s.shape, "COEF", coef_yev.shape, "COEFR", coef_yev_repeated.shape)
theta_s_withcoeff = tf.multiply(theta_s, coef_yev_repeated)
result['loss_m_to_common'] = tf.reduce_sum(theta_s_withcoeff) * lambda_

# LOSS COMMON TO 0
experts_common_to_1 = self.all_ratings.aggregate_index * tf.ones(shape=objects_common_to_1.shape,
                                                                 dtype=tf.int64)
idx_common_to_1 = tf.stack((experts_common_to_1, objects_common_to_1), axis=1)
s_qv_common_to_1 = self.all_ratings.model(idx_common_to_1)

sm1 = tf.math.square(s_qv_common_to_1 - 1)

# print(idx_common_to_1, s_qv_common_to_1, sm1)

result['loss_common_to_1'] = tf.reduce_sum(sm1) * mu

# TOTAL LOSS
result['loss'] = result['loss_fit'] + result['loss_m_to_common'] + \
                 result['loss_common_to_1']

In [ ]:
pd.DataFrame(result)

In [ ]:
video_info_keys = ['uploader', 'language', 'views', 'publication_date', 'name']
video_info = {v.video_id: {k: getattr(v, k) for k in video_info_keys} for v in Video.objects.all()}

objs = l.all_ratings.objects
predictions = [m(objs) for m in l.aggregator.models] + [l.aggregator(objs)]

experts = [UserPreferences.objects.get(id=x).username if isinstance(x, int) else x for x in l.all_ratings.experts]

F = l.all_ratings.output_features + ['sum']

for exp_name, preds in zip(experts, predictions):
    suff = ["_vid", "_pred"] + [f"_{k}" for k in video_info_keys]
    best_worst = {f + s: [] for s in suff for f in F}
    
    preds_ = np.zeros(shape=(preds.shape[0], preds.shape[1] + 1))
    preds_[:, :-1] = preds
    preds_[:, -1] = np.sum(preds, axis=1)
    
    for i, f in enumerate(F):
        out_arr = preds_[:, i]
        idx_sorted = np.argsort(out_arr)[::-1]
        
        preds_sorted = out_arr.copy()[idx_sorted]
        objs_sorted = np.array(objs)[idx_sorted]
        
        
        for idx in range(5):
            best_worst[f"{f}_vid"].append(objs_sorted[idx])
            for k in video_info_keys:
                best_worst[f"{f}_{k}"].append(video_info[objs_sorted[idx]][k])
            best_worst[f"{f}_pred"].append(preds_sorted[idx])
            
        for s in suff:
            best_worst[f"{f}{s}"].append("...")
            
        for idx in range(len(idx_sorted))[-5:]:
            best_worst[f"{f}_vid"].append(objs_sorted[idx])
            for k in video_info_keys:
                best_worst[f"{f}_{k}"].append(video_info[objs_sorted[idx]][k])
            best_worst[f"{f}_pred"].append(preds_sorted[idx])
        
    df = pd.DataFrame(best_worst)
    df.to_csv(f'{exp_name}.csv', index=False)

In [ ]:
pd.DataFrame(best_worst)

In [ ]:
np.where(idx_common[:, 1] == 44)

In [ ]:
coef_yev_repeated[44]

In [ ]:
plt.hist(num_float)

In [ ]:
result

In [ ]:
x = np.linspace(-10, 10, 100)
plt.plot(x, np.log(1 + np.exp(x)))